In [2]:
import bw2data as bd
import bw2io as bi

In [5]:
assert bd.__version__ > (4, 0), "This notebook is only for Brightway 2.5"

In [7]:
bd.projects.set_current("LC IMPACT case study")

# Create default `biosphere` database and LCIA methods

In [8]:
bi.create_default_biosphere3()
bi.create_core_migrations()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/22/2022 21:59:11
  Finished: 04/22/2022 21:59:11
  Total time elapsed: 00:00:00
  CPU %: 86.30
  Memory %: 1.27
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 2.01 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



# Import `ecoinvent 3.8` with the `cutoff` system model

In [11]:
ei = bi.SingleOutputEcospold2Importer("/Users/cmutel/Documents/LCA/Ecoinvent/3.8/cutoff/datasets", "ecoinvent")
ei.apply_strategies()
ei.statistics()
assert ei.all_linked

Extracting XML data from 19565 datasets
Extracted 19565 datasets in 142.00 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy:

In [12]:
ei.write_database()

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:07


Title: Writing activities to SQLite3 database:
  Started: 04/22/2022 22:10:14
  Finished: 04/22/2022 22:11:22
  Total time elapsed: 00:01:07
  CPU %: 85.30
  Memory %: 21.61
Created database: ecoinvent


Brightway2 SQLiteBackend: ecoinvent

# Create database for our functional unit

We now define our functional unit. We will look at one passenger kilometer of transport in a modern auto (EURO 5 standard), with three fuel possibilities:

* Low-sulfur petrol
* E85 from Brazilian sugarcane
* E85 from American maize

Metastudies ([1](http://doi.org/10.1021/es305209a),[2](http://doi.org/10.1021/es502495u)) on the effective substitution ratio of ethanol compared to petrol are inconclusive. We therefore assume that ethanol substitutes 1:1 for petrol on an energetic basis in modern, fuel-injected cars.

Ethanol has a lower energy density than petrol, therefor *1* kg of petrol is equivalent to *1.46* kg of ethanol

Emissions from the combustion of ethanol differ from low-sulfur petrol. However, this difference is difficult to summarize, as it is dependent on, among other things, engine type, driving patterns, and
climate ([3](http://doi.org/10.1016/j.rser.2013.03.046), [4](http://doi.org/10.1016/j.rser.2013.09.022), [5](http://doi.org/10.1177/0954407011407254)). Given this uncertainty, we make the following rough estimates:

* Ethanol has effectively no sulfur, so no SO2 is emitted.
* Emissions of CO decrease 10 percent
* Hydrocarbon emissions decrease 10 percent

The easiest way to define the functional unit was in the separate spreadsheet `"ethanol-inventory.xlsx"`, which is included in the supporting information.

In [15]:
imp = bi.ExcelImporter("ethanol-inventory.xlsx")
imp.apply_strategies()
imp.match_database("ecoinvent", fields=('name', 'unit', 'location'))
imp.match_database(fields=['name'])
assert imp.all_linked

/Users/cmutel/miniconda3/envs/bw25reg/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Extracted 1 worksheets in 0.08 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.28 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
6 datasets
70 exchanges
0 unlinked exchanges
  


In [16]:
imp.write_database()

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/22/2022 22:14:48
  Finished: 04/22/2022 22:14:48
  Total time elapsed: 00:00:00
  CPU %: 36.50
  Memory %: 21.99
Created database: Fuel comparison
